# Cell Mutation and Parameter Studies

In this notebook you will learn how to:
- Mutate cell properties (material, density, importance, name)
- Fill and unfill cells with universes
- Verify that mutations survive geometry rebuilds
- Run a simple parameter study by varying shield density

In [2]:
%matplotlib inline
import aleathor as ath
import matplotlib.pyplot as plt

In [3]:
# Build a simple sphere-in-box model
model = ath.Model("Mutation Demo")

sphere = ath.Sphere(0, 0, 0, radius=5)
box    = ath.Box(-10, 10, -10, 10, -10, 10)

model.add_cell(region=-sphere,            material=1, density=10.0, name="core")
model.add_cell(region=-box & +sphere,     material=2, density=7.8,  name="shield")

for c in model.cells:
    print(f"Cell {c.id}: mat={c.material}, rho={c.density}, name={c.name!r}")

Cell 1: mat=1, rho=10.0, name='core'
Cell 2: mat=2, rho=7.8, name='shield'


## Mutate cell properties

Use property setters directly on the `Cell` object. Changes mark the model dirty so the C system is rebuilt on the next query.

In [4]:
cell = model[2]  # the shield cell

print("Before:")
print(f"  material   = {cell.material}")
print(f"  density    = {cell.density}")
print(f"  importance = {cell.importance}")
print(f"  name       = {cell.name!r}")

cell.material   = 5
cell.density    = 8.0
cell.importance = 0.5
cell.name       = "tungsten_shield"

# Re-fetch to confirm (reads from the rebuilt C system)
cell = model[2]
print("\nAfter:")
print(f"  material   = {cell.material}")
print(f"  density    = {cell.density}")
print(f"  importance = {cell.importance}")
print(f"  name       = {cell.name!r}")

Before:
  material   = 2
  density    = 7.8
  importance = 1.0
  name       = 'shield'

After:
  material   = 5
  density    = 8.0
  importance = 0.5
  name       = 'tungsten_shield'


## Fill a cell with a universe

`fill_with(universe_id)` sets the FILL, `unfill()` removes it.

In [4]:
cell = model[2]

cell.fill_with(3)
print(f"fill = {cell.fill}, is_filled = {cell.is_filled}")

cell.unfill()
print(f"fill = {cell.fill}, is_filled = {cell.is_filled}")

fill = 3, is_filled = True
fill = None, is_filled = False


## Mutations survive rebuild

When a property setter marks the model dirty, the next query triggers a full rebuild from the Python `_CellData` objects. Fills set via `fill_with()` are persisted in the Python layer, so they survive the rebuild.

In [5]:
cell = model[2]
cell.fill_with(7)          # sets fill in Python + C
cell.material = 99         # marks model dirty → triggers rebuild on next read

# After the implicit rebuild, fill should still be 7
cell = model[2]
print(f"material = {cell.material}  (expected 99)")
print(f"fill     = {cell.fill}  (expected 7)")

material = 99  (expected 99)
fill     = 7  (expected 7)


## Parameter study — shield density vs. areal density

We trace a ray through the model at several shield densities and plot the total areal density
(density × path length) through the shield material.

In [ ]:
# Reset material back to original for the study
cell = model[2]
cell.material = 2
cell.unfill()

densities = [2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0]
areal_densities = []

for rho in densities:
    model[2].density = rho
    trace = model.trace(start=(-10, 0, 0), end=(10, 0, 0))
    # Sum density * length for shield segments
    areal = sum(s.density * s.length
               for s in trace if s.material == 2 and s.length < 1e10)
    areal_densities.append(areal)

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(densities, areal_densities, 'o-')
ax.set_xlabel('Shield density (g/cm³)')
ax.set_ylabel('Areal density (g/cm²)')
ax.set_title('Shield areal density vs. bulk density')
ax.grid(True, alpha=0.3)
plt.tight_layout()